<a href="https://colab.research.google.com/github/Maviya5/CE888/blob/master/Assignment%202/Google_Analysis_covariate_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import os
import json
from pandas.io.json import json_normalize

In [11]:
import os

os.environ['KAGGLE_USERNAME'] = "namnitha" # username from the json file

os.environ['KAGGLE_KEY'] = "90ceb5b09eaa184f556d4577eb017b49" # key from the json file

!kaggle competitions download -c ga-customer-revenue-prediction # api copied from kaggle

sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
 82% 50.0M/61.1M [00:01<00:00, 37.6MB/s]
100% 61.1M/61.1M [00:01<00:00, 48.7MB/s]
train_v2.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission_v2.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_v2.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [12]:
!ls

sample_data		      test.csv	       train.csv.zip
sample_submission.csv.zip     test.csv.zip     train_v2.csv.zip
sample_submission_v2.csv.zip  test_v2.csv.zip


In [13]:
from zipfile import ZipFile
train = "train.csv.zip"
with ZipFile(train,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [14]:
train = pd.read_csv('train.csv', low_memory=True)
test = pd.read_csv('test.csv', low_memory=True)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Loading the train and test data



In [0]:
#Define function to load data (Kudos to Julian for that)
def load_df(csv_path='train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [17]:
print(train.head())

  channelGrouping      date  ... visitNumber visitStartTime
0  Organic Search  20160902  ...           1     1472830385
1  Organic Search  20160902  ...           1     1472880147
2  Organic Search  20160902  ...           1     1472865386
3  Organic Search  20160902  ...           1     1472881213
4  Organic Search  20160902  ...           2     1472822600

[5 rows x 12 columns]


In [18]:
print(test.head())

  channelGrouping      date  ... visitNumber visitStartTime
0  Organic Search  20171016  ...           2     1508151024
1  Organic Search  20171016  ...           1     1508175522
2  Organic Search  20171016  ...           1     1508143220
3  Organic Search  20171016  ...           1     1508193530
4  Organic Search  20171016  ...           1     1508217442

[5 rows x 12 columns]


In [0]:
train['geoNetwork.country'].hist() 
test['geoNetwork.country'].hist()
train["trafficSource.adContent"].hist() 
test['trafficSource.adContent'].hist()
train["trafficSource.source"].hist() 
test['trafficSource.source'].hist()

In [0]:
train['device.browserVersion'].hist() 
test['device.browserVersion'].hist()
train["device.deviceCategory"].hist() 
test['device.deviceCategory'].hist()
train["device.flashVersion"].hist() 
test['device.flashVersion'].hist()

In [19]:
#Impute missing values (mean for numeric, mode for categorical)
print('Imputing missing values...')
for i in train.columns:
    if train[i].dtype == 'object':
      train[i] = train[i].fillna(train[i].mode().iloc[0])
    elif (train[i].dtype != 'object'):
      train[i] = train[i].fillna(np.mean(train[i]))


for i in test.columns:
    if test[i].dtype == 'object':
      test[i] = test[i].fillna(test[i].mode().iloc[0])
    elif (test[i].dtype != 'object'):
      test[i] = test[i].fillna(np.mean(test[i]))
    

print('Nulls in train set:', train.isnull().sum().sum())
print('Nulls in test set:', test.isnull().sum().sum())

Imputing missing values...
Nulls in train set: 0
Nulls in test set: 0


In [20]:
## label encode categorical variables
print('Label Encoding categorical variables...')
for col in train.columns:
    if train[col].dtype == 'object':
      train[col] = train[col].astype('category')
      train[col] = train[col].cat.codes

for col in test.columns:
    if test[col].dtype == 'object':
      test[col] = test[col].astype('category')
      test[col] = test[col].cat.codes

Label Encoding categorical variables...


In [22]:
print('train shape:', train.shape)
print('test shape:', test.shape)

train shape: (903653, 13)
test shape: (804684, 13)


In [0]:
## Creating a dummy y label and drop the target variable
train['set'] = 0
test['set'] = 1


In [24]:
train_df = train.sample(10000, random_state=697)
test_df = test.sample(10000, random_state=466)

all_data = train_df.append(test_df)
y_label = all_data['set']
all_data = all_data.drop('set',axis=1)

#Make sure the new dataframe contains all the initial features
print('New dataframe shape:', all_data.shape)

New dataframe shape: (20000, 12)


In [25]:
model = RandomForestClassifier(n_estimators = 50, max_depth = 5, min_samples_leaf = 5)
feat_to_drop = []
for col in all_data.columns:
    score = cross_val_score(model,pd.DataFrame(all_data[col]),y_label,cv=4,scoring='roc_auc')
    if np.mean(score) > 0.8:
        feat_to_drop.append(col)
    print(col,np.mean(score))

channelGrouping 0.57437524
date 1.0
device 0.9936929200000001
fullVisitorId 0.5668589199999999
geoNetwork 0.67790168
sessionId 0.55659358
socialEngagementType 0.5
totals 0.842399
trafficSource 0.9986791799999999
visitId 1.0
visitNumber 0.5180248599999999
visitStartTime 1.0


In [26]:
print('Number of features with covariate shift:', len(feat_to_drop))

Number of features with covariate shift: 6


For the covariate shift detection; the auc-roc score of the feature should be greater that 0.80. if its is less than 0.80 then there is no covariate sift in features
  As a result for this data set there are 6 columns that have covariate shit